# Converstational Fine-Tuning

[From Unsloth Documentation](https://docs.unsloth.ai/basics/tutorial-how-to-finetune-llama-3-and-use-in-ollama#id-9.-customizable-chat-templates).

Create a custom chatbot by finetuning Llama-3 with [Unsloth](https://github.com/unslothai/unsloth). It can run locally via Ollama on your PC.

Unsloth makes finetuning much easier, and can automatically export the finetuned model to Ollama with integrated automatic `Modelfile` creation!

## Selecting a model to finetune

Let's now select a model for finetuning! We defaulted to Llama-3 from Meta / Facebook which was trained on a whopping 15 trillion "tokens". Unsloth supports these models and more! In fact, simply type a model from the Hugging Face model hub to see if it works! We'll error out if it doesn't work.

In [ ]:
from unsloth import FastLanguageModel
from transformers import TextStreamer

There are 3 other settings which you can toggle:
- `max_seq_length = 2048`: This determines the context length of the model. Gemini for example has over 1 million context length, whilst Llama-3 has 8192 context length. We allow you to select ANY number - but we recommend setting it 2048 for testing purposes. Unsloth also supports very long context finetuning, and we show we can provide 4x longer context lengths than the best.
- `dtype = None`: Keep this as None, but you can select torch.float16 or torch.bfloat16 for newer GPUs.
- `load_in_4bit = True`: We do finetuning in 4 bit quantization. This reduces memory usage by 4x, allowing us to actually do finetuning in a free 16GB memory GPU. 4 bit quantization essentially converts weights into a limited set of numbers to reduce memory usage. A drawback of this is there is a 1-2% accuracy degradation. Set this to False on larger GPUs like H100s if you want that tiny extra accuracy.

In [3]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

In [4]:
# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
model, tokenizer = FastLanguageModel.from_pretrained(
    # More models at https://huggingface.co/unsloth
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    # model_name = "unsloth/Llama-3.2-1B-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


## PEFT & Parameters for finetuning

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

Now to customize your finetune, you can edit the numbers below. We recommend keeping the defaults for now, but you can change them if you want to **experiment**. The goal is to change these numbers to increase accuracy, but also counteract over-fitting. Over-fitting is when you make the language model memorize a dataset, and not be able to answer novel new questions. We want to a final model to answer unseen questions, and not do memorization.

- `r`: Choose any number > 0 ! Suggested 8, 16, 32, 64, 128. The rank of the finetuning process. A larger number uses more memory and will be slower, but can increase accuracy on harder tasks. We normally suggest numbers like 8 (for fast finetunes), and up to 128. Too large numbers can causing over-fitting, damaging your model's quality.
- `target_modules`: We select all modules to finetune. You can remove some to reduce memory usage and make training faster, but we highly do not suggest this. Just train on all modules!
- `lora_alpha`: The scaling factor for finetuning. A larger number will make the finetune learn more about your dataset, but can promote over-fitting. We suggest this to equal to the rank r, or double it.
- `lora_dropout`: Supports any, but = 0 is optimized. Leave this as 0 for faster training! Can reduce over-fitting, but not that much.
- `bias`: Supports any, but = "none" is optimized. Leave this as 0 for faster and less over-fit training!
- `use_gradient_checkpointing`: True or "unsloth" for very long context. Options include True, False and "unsloth". We suggest "unsloth" since we reduce memory usage by an extra 30% and support extremely long context finetunes.You can read up [here](https://unsloth.ai/blog/long-context) for more details.
- `random_state`: The number to determine deterministic runs. Training and finetuning needs random numbers, so setting this number makes experiments reproducible.
- `use_rslora`: We support rank stabilized LoRA. Advanced feature to set the lora_alpha = 16 automatically. You can use this if you want!
- `loftq_config`: Advanced feature to initialize the LoRA matrices to the top r singular vectors of the weights. Can improve accuracy somewhat, but can make memory usage explode at the start.

In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"
    ],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.2.15 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## Alpaca Dataset

We will now use the [Alpaca Dataset](https://huggingface.co/datasets/vicgalle/alpaca-gpt4) created by calling GPT-4 itself. It is a list of 52,000 instructions and outputs which was very popular when Llama-1 was released, since it made finetuning a base LLM be competitive with ChatGPT itself.

In [9]:
from datasets import load_dataset

In [10]:
dataset = load_dataset("vicgalle/alpaca-gpt4", split="train")
print(dataset.column_names)

['instruction', 'input', 'output', 'text']


In [11]:
# print out the first example in the dataset: instruction, input, output, text
example_id = 1
print(f"# Instruction:\n{dataset['instruction'][example_id]}\n")
print(f"# Input:\n{dataset['input'][example_id]}\n")
print(f"# Output:\n{dataset['output'][example_id]}\n")
print(f"# Text:\n{dataset['text'][example_id]}\n")

# Instruction:
What are the three primary colors?

# Input:


# Output:
The three primary colors are red, blue, and yellow. These colors are called primary because they cannot be created by mixing other colors and all other colors can be made by combining them in various proportions. In the additive color system, used for light, the primary colors are red, green, and blue (RGB).

# Text:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
What are the three primary colors?

### Response:
The three primary colors are red, blue, and yellow. These colors are called primary because they cannot be created by mixing other colors and all other colors can be made by combining them in various proportions. In the additive color system, used for light, the primary colors are red, green, and blue (RGB).



### Merge multiple columns

You can see there are 3 columns in each row - an instruction, and input and an output. We essentially combine each row into 1 large prompt like below. We then use this to finetune the language model, and this made it very similar to ChatGPT. We call this process **supervised instruction finetuning**.

One issue is this dataset has multiple columns. For `Ollama` and `llama.cpp` to function like a custom `ChatGPT` Chatbot, **we must only have 2 columns** - an `instruction` and an `output` column.

To merge multiple columns into 1, use `merged_prompt`.
* Enclose all columns in curly braces `{}`.
* Optional text must be enclused in `[[]]`. For example if the column "Pclass" is empty, the merging function will not show the text and skp this. This is useful for datasets with missing values.
* You can select every column, or a few!
* Select the output or target / prediction column in `output_column_name`. For the Alpaca dataset, this will be `output`.

**Multi turn conversations**: To make the finetune handle multiple turns (like in ChatGPT), we have to create a "fake" dataset with multiple turns - we use `conversation_extension` to randomnly select some conversations from the dataset, and pack them together into 1 conversation. For example, if you set it to 3, we randomly select 3 rows and merge them into 1! Setting them too long can make training slower, but could make your chatbot and final finetune much better!

![Conversation Extension](misc/conversation_extension.jpg "Conversation Extension")

In [12]:
from unsloth import to_sharegpt

In [13]:
dataset = to_sharegpt(
    dataset,
    merged_prompt="{instruction}[[\nYour input is:\n{input}]]",
    output_column_name="output",
    conversation_extension=3,  # Select more to handle longer conversations
)

Finally use `standardize_sharegpt` to fix up the dataset!

In [14]:
from unsloth import standardize_sharegpt

In [15]:
dataset = standardize_sharegpt(dataset)

### Customizable Chat Templates

You also need to specify a chat template. Previously, you could use the Alpaca format as shown below

```python
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""
```

The issue is the Alpaca format has 3 fields, whilst OpenAI style chatbots must only use 2 fields (instruction and response). That's why we used the `to_sharegpt` function to merge these columns into 1.

But may be a bad idea because ChatGPT style finetunes require only 1 prompt? Since we successfully merged all dataset columns into 1 using Unsloth, we just require you must put a `{INPUT}` field for the instruction and an `{OUTPUT}` field for the model's output field. We in fact allow an optional {SYSTEM} field as well which is useful to customize a system prompt just like in ChatGPT.

You can also not put a `{SYSTEM}` field, and just put plain text.

```python
chat_template = """{SYSTEM}
USER: {INPUT}
ASSISTANT: {OUTPUT}"""
```
**For the ChatML format**:

In [16]:
from unsloth import apply_chat_template

In [17]:
# You can observe that tokenizer chat template is empty (we are using a base model)
tokenizer.chat_template

In [ ]:
chat_template = """<|im_start|>system
{SYSTEM}<|im_end|>
<|im_start|>user
{INPUT}<|im_end|>
<|im_start|>assistant
{OUTPUT}<|im_end|>"""

dataset = apply_chat_template(
    dataset,
    tokenizer=tokenizer,
    chat_template=chat_template,
    # default_system_message = "You are a helpful assistant", << [OPTIONAL]
)

In [ ]:
# You can observe that tokenizer chat template got updated!
tokenizer.chat_template

"{{ bos_token }}{% if messages[0]['role'] == 'system' %}{{ '<|im_start|>system\n' + messages[0]['content'] + '<|im_end|>' }}{% set loop_messages = messages[1:] %}{% else %}{{ '<|im_start|>system\nBelow are some instructions that describe some tasks. Write responses that appropriately complete each request.<|im_end|>' }}{% set loop_messages = messages %}{% endif %}{% for message in loop_messages %}{% if message['role'] == 'user' %}{{ '\n<|im_start|>user\n' + message['content'] + '<|im_end|>' }}{% elif message['role'] == 'assistant' %}{{ '\n<|im_start|>assistant\n' + message['content'] + '<|im_end|><|end_of_text|>' }}{% else %}{{ raise_exception('Only user and assistant roles are supported!') }}{% endif %}{% endfor %}{% if add_generation_prompt %}{{ '\n<|im_start|>assistant\n' }}{% endif %}"

In [35]:
dataset

Dataset({
    features: ['conversations', 'text'],
    num_rows: 52002
})

In [50]:
# get an example from the dataset
example = dataset[14]
print(f"Example:\n{example['text']}")

Example:
<|begin_of_text|><|im_start|>system
Below are some instructions that describe some tasks. Write responses that appropriately complete each request.<|im_end|>
<|im_start|>user
Classify the following into animals, plants, and minerals
Your input is:
Oak tree, copper ore, elephant<|im_end|>
<|im_start|>assistant
Animals: Elephant
Plants: Oak tree
Minerals: Copper ore<|im_end|><|end_of_text|>
<|im_start|>user
Take the following text and add two more sentences to it.
Your input is:
Alice and Bob were two best friends living in the same city.<|im_end|>
<|im_start|>assistant
Alice and Bob were two best friends living in the same city. They met each other in high school and have been inseparable ever since. Over the years, they have shared countless memories and adventures together.<|im_end|><|end_of_text|>
<|im_start|>user
Convert this list of numbers into a comma-separated string
Your input is:
[10, 20, 30, 40, 50]<|im_end|>
<|im_start|>assistant
"10, 20, 30, 40, 50"<|im_end|><|end_

## Inference: Before Finetuning

In [21]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
print("Inference enabled!")

Inference enabled!


In [22]:
messages = [ # Change below!
    {"role": "user", "content": "Continue the fibonacci sequence! Your input is 1, 1, 2, 3, 5, 8,"},
]

In [23]:
formatted_msg = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    tokenize = False,
)
print(formatted_msg)

<|begin_of_text|><|im_start|>system
Below are some instructions that describe some tasks. Write responses that appropriately complete each request.<|im_end|>
<|im_start|>user
Continue the fibonacci sequence! Your input is 1, 1, 2, 3, 5, 8,<|im_end|>
<|im_start|>assistant



In [25]:
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")


text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

The Fibonacci sequence is a sequence of numbers where the next number is the sum of the previous two numbers. For example, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144,... The Fibonacci sequence is named after Leonardo of Pisa, also known as Fibonacci. The Fibonacci sequence is an example of a recursive sequence, which means that the next number in the sequence is defined in terms of the previous two numbers. In this case, the next number is the sum of the previous two numbers. The Fibonacci sequence has many


## Train the model

Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [25]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

In [65]:
FastLanguageModel.for_training(model)
print("Ready to train!")

Ready to train!


We do not normally suggest changing the parameters, but to elaborate on some of them:

- `per_device_train_batch_size`: Increase the batch size if you want to utilize the memory of your GPU more. Also increase this to make training more smooth and make the process not over-fit. We normally do not suggest this, since this might make training actually slower due to padding issues. We normally instead ask you to increase gradient_accumulation_steps which just does more passes over the dataset.
- `gradient_accumulation_steps`: Equivalent to increasing the batch size above itself, but does not impact memory consumption! We normally suggest people increasing this if you want smoother training loss curves.
- `max_steps` or `num_train_epochs`: We can set steps to 60 for faster training. For full training runs which can take hours, instead comment out max_steps, and replace it with num_train_epochs = 1. Setting it to 1 means 1 full pass over your dataset. We normally suggest 1 to 3 passes, and no more, otherwise you will over-fit your finetune.
- `learning_rate`: Reduce the learning rate if you want to make the finetuning process slower, but also converge to a higher accuracy result most likely. We normally suggest 2e-4, 1e-4, 5e-5, 2e-5 as numbers to try.

In [ ]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 8,
        warmup_steps = 5,
        #max_steps = -1,  # 60 steps is a good start for testing
        num_train_epochs = 1, # For longer training runs!
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

In [ ]:
trainer_stats = trainer.train()

### Saving finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [31]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")  # Local saving
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

### Load the model back

Now if you want to load the LoRA adapters we just saved:

In [26]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


## Inference: After Finetuning

Let's run the model! Unsloth makes inference natively 2x faster as well! You should **use prompts which are similar to the ones you had finetuned on, otherwise you might get bad results**!

In [27]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
print("Inference mode!")

Inference mode!


In [28]:
messages = [ # Change below!
    {"role": "user", "content": "Continue the fibonacci sequence! Your input is 1, 1, 2, 3, 5, 8,"},
]

In [29]:
formatted_msg = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    tokenize = False,
)
print(formatted_msg)

<|begin_of_text|><|im_start|>system
Below are some instructions that describe some tasks. Write responses that appropriately complete each request.<|im_end|>
<|im_start|>user
Continue the fibonacci sequence! Your input is 1, 1, 2, 3, 5, 8,<|im_end|>
<|im_start|>assistant



In [32]:
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")


text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

Your input is 1, 1, 2, 3, 5, 8. The next number in the Fibonacci sequence is 13.<|im_end|><|end_of_text|>


Since we created an actual chatbot, you can also do longer conversations by manually adding alternating conversations between the user and assistant!

In [57]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [                         # Change below!
    {"role": "user",      "content": "Continue the fibonacci sequence! Your input is 1, 1, 2, 3, 5, 8"},
    {"role": "assistant", "content": "The fibonacci sequence continues as 13, 21, 34, 55 and 89."},
    {"role": "user",      "content": "What is France's tallest tower called?"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

The tallest tower in France is the Eiffel Tower, which stands at a height of 324 meters (1,063 feet). It was built in 1889 for the World Fair, and is located in the city of Paris.<|im_end|><|end_of_text|>


## Exporting to Ollama

Finally we can export our finetuned model to Ollama itself! [Unsloth](https://github.com/unslothai/unsloth) now allows you to automatically finetune and create a [Modelfile](https://github.com/ollama/ollama/blob/main/docs/modelfile.md), and export to [Ollama](https://ollama.com/)! This makes finetuning much easier and provides a seamless workflow from `Unsloth` to `Ollama`!

We shall save the model to GGUF / llama.cpp

We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

We also support saving to multiple GGUF options in a list fashion! This can speed things up by 10 minutes or more if you want multiple export formats! Head over to https://github.com/ggerganov/llama.cpp to learn more about GGUF. We also have some manual instructions of how to export to GGUF if you want here: https://github.com/unslothai/unsloth/wiki#manually-saving-to-gguf

In [ ]:
# Save to 8bit Q8_0
if True: model.save_pretrained_gguf("model", tokenizer)

# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

`Ollama` needs a `Modelfile`, which specifies the model's prompt format. Let's print Unsloth's auto generated one:

In [ ]:
print(tokenizer._ollama_modelfile)

We now will create an `Ollama` model called `unsloth_model` using the `Modelfile` which we auto generated!

```bash
ollama create unsloth_model -f ./model/Modelfile
```

And we can now call the model for inference if you want to do call the Ollama server itself which is running on your own local machine / in the free Colab notebook in the background. Remember you can edit the yellow underlined part.

```bash
curl http://localhost:11434/api/chat -d '{ \
    "model": "unsloth_model", \
    "messages": [ \
        { "role": "user", "content": "Continue the Fibonacci sequence: 1, 1, 2, 3, 5, 8," } \
    ] \
}'
```